In [3]:
import pandas as pd
import numpy as np

In [47]:
data=pd.read_csv('CCMLEmployeeData.csv')
events = pd.read_excel("Event.xlsx")
target_names = events.Type.unique().tolist()
new_df= pd.read_csv('Events for Recommendation.csv')

In [5]:

events['category_id'] = events['Type'].factorize()[0]

In [6]:
category_id_df = events[['Type', 'category_id']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'Type']].values)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(events.Events).toarray()
labels = events.Type
features.shape

(362, 121)

In [53]:

from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

models = [
    LinearSVC(),  
]
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])
print(cv_df)


C:\Users\HP\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


  model_name  fold_idx  accuracy
0  LinearSVC         0  0.214286
1  LinearSVC         1  0.188889
2  LinearSVC         2  0.279412
3  LinearSVC         3  0.360000
4  LinearSVC         4  0.333333


In [22]:
model = LinearSVC()

X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features, labels, events.index, test_size=0.15, random_state=0)

In [23]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [26]:
from sklearn.metrics import confusion_matrix

conf_mat = confusion_matrix(y_test, y_pred)


In [27]:
conf_mat

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [28]:
model.fit(features, labels)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [43]:
new_df

,Events
0,Attend the Android application development wor...
1,Lockdown special courses on Ydemi. 22 hours left!
2,Machine learning is new technology attend the ...
3,natural language processing is the heart of ev...
4,Attend areas of Blockchain seminar
5,50 % off on all Machine Learning and Deep lear...
6,Arranged Machine Learning workshop and Internship
7,Digital Marketing workshop and Webinar
8,Arrange a Machine Learning Hackathon
9,Workshops for Higher Education of students


In [51]:
text_features = tfidf.transform(new_df)
predictions = model.predict(text_features)

for key, val in zip(new_df, predictions):
    new_df = new_df.append({'Event' : key , 'Recommended Employees' : ', '.join(val)},ignore_index=True)
new_df = new_df[['Event','Recommended Employees']]
new_df.to_excel('aala.xlsx',index=False)


In [52]:
new_df

,Event,Recommended Employees
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN
9,NaN,NaN
